In [1]:
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType,BooleanType,DateType,IntegerType
from pyspark.ml import Pipeline
from pyspark.ml.feature import (
    OneHotEncoder,
    StringIndexer,
    VectorAssembler,
    VectorIndexer,
    MinMaxScaler
)

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1617492240956_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
spark = SparkSession \
    .builder \
    .appName("rdtest") \
    .enableHiveSupport() \
    .getOrCreate()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
df = spark.sql("SELECT * from rdtest2.rdtest2 where aggdate like '2018-01-%'")
df.head(3)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(date=Row(utc='2018-01-01T22:44:22.000Z', local='2018-01-02T09:44:22+11:00'), parameter='pm25', location='Smithton', value=0.0, unit='µg/m³', city='Tasmania Region', attribution=[Row(name='Environmental Protection Authority - Tasmania', url='http://epa.tas.gov.au')], averagingperiod=Row(value=0.25, unit='hours'), coordinates=Row(latitude=-40.8505, longitude=145.133), country='AU', sourcename='Australia - Tasmania', sourcetype='government', mobile=False, aggdate='2018-01-02'), Row(date=Row(utc='2018-01-01T22:44:22.000Z', local='2018-01-02T09:44:22+11:00'), parameter='pm10', location='Smithton', value=0.0, unit='µg/m³', city='Tasmania Region', attribution=[Row(name='Environmental Protection Authority - Tasmania', url='http://epa.tas.gov.au')], averagingperiod=Row(value=0.25, unit='hours'), coordinates=Row(latitude=-40.8505, longitude=145.133), country='AU', sourcename='Australia - Tasmania', sourcetype='government', mobile=False, aggdate='2018-01-02'), Row(date=Row(utc='2018-01-01T22

In [4]:
df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- date: struct (nullable = true)
 |    |-- utc: string (nullable = true)
 |    |-- local: string (nullable = true)
 |-- parameter: string (nullable = true)
 |-- location: string (nullable = true)
 |-- value: double (nullable = true)
 |-- unit: string (nullable = true)
 |-- city: string (nullable = true)
 |-- attribution: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- url: string (nullable = true)
 |-- averagingperiod: struct (nullable = true)
 |    |-- value: double (nullable = true)
 |    |-- unit: string (nullable = true)
 |-- coordinates: struct (nullable = true)
 |    |-- latitude: double (nullable = true)
 |    |-- longitude: double (nullable = true)
 |-- country: string (nullable = true)
 |-- sourcename: string (nullable = true)
 |-- sourcetype: string (nullable = true)
 |-- mobile: boolean (nullable = true)
 |-- aggdate: string (nullable = true)

## Transformations

* Drop date
* Encode parameter
* Encode location
* Scale value
* Drop unit
* Encode city
* Drop attribution
* Drop averaging period
* Drop coordinates
* Encode country
* Encode source name
* Encode source type
* Convert mobile to integer
* Featurize aggdate
* Add label for good/bad air quality

In [5]:
# Drop columns
df = df.drop('date').drop('unit').drop('attribution').drop('averagingperiod').drop('coordinates')
df.head(3)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(parameter='pm25', location='Smithton', value=3.0, city='Tasmania Region', country='AU', sourcename='Australia - Tasmania', sourcetype='government', mobile=False, aggdate='2018-01-06'), Row(parameter='pm10', location='Smithton', value=3.0, city='Tasmania Region', country='AU', sourcename='Australia - Tasmania', sourcetype='government', mobile=False, aggdate='2018-01-06'), Row(parameter='pm25', location='Wynyard', value=2.0, city='Tasmania Region', country='AU', sourcename='Australia - Tasmania', sourcetype='government', mobile=False, aggdate='2018-01-06')]

In [7]:
# Mobile field to int
from pyspark.sql.functions import col
df = df.withColumn("ismobile",col("mobile").cast(IntegerType())).drop('mobile')
df.head(3)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(parameter='pm25', location='Smithton', value=2.0, city='Tasmania Region', country='AU', sourcename='Australia - Tasmania', sourcetype='government', aggdate='2018-01-09', ismobile=0), Row(parameter='pm10', location='Smithton', value=9.0, city='Tasmania Region', country='AU', sourcename='Australia - Tasmania', sourcetype='government', aggdate='2018-01-09', ismobile=0), Row(parameter='pm25', location='Wynyard', value=0.0, city='Tasmania Region', country='AU', sourcename='Australia - Tasmania', sourcetype='government', aggdate='2018-01-09', ismobile=0)]

In [10]:
# scale value
from pyspark.ml.feature import StandardScaler

value_assembler = VectorAssembler(inputCols=["value"], 
                            outputCol="value_vec")
value_scaler = StandardScaler(inputCol="value_vec", outputCol="value_scaled")
value_pipeline = Pipeline(stages=[value_assembler, value_scaler])
value_model = value_pipeline.fit(df)
xform_df = value_model.transform(df)
xform_df.head(3)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(parameter='pm25', location='Smithton', value=1.0, city='Tasmania Region', country='AU', sourcename='Australia - Tasmania', sourcetype='government', aggdate='2018-01-07', ismobile=0, value_vec=DenseVector([1.0]), value_scaled=DenseVector([0.0001])), Row(parameter='pm10', location='Smithton', value=1.0, city='Tasmania Region', country='AU', sourcename='Australia - Tasmania', sourcetype='government', aggdate='2018-01-07', ismobile=0, value_vec=DenseVector([1.0]), value_scaled=DenseVector([0.0001])), Row(parameter='pm25', location='Wynyard', value=3.0, city='Tasmania Region', country='AU', sourcename='Australia - Tasmania', sourcetype='government', aggdate='2018-01-07', ismobile=0, value_vec=DenseVector([3.0]), value_scaled=DenseVector([0.0003]))]

In [11]:
# featurize date
from pyspark.sql.functions import unix_timestamp, to_date
xform_df = xform_df.withColumn('aggdt', 
                   to_date(unix_timestamp(col('aggdate'), 'yyyy-MM-dd').cast("timestamp")))
xform_df.head(3)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(parameter='pm10', location='AD0942A', value=26.0, city='Escaldes-Engordany', country='AD', sourcename='EEA Andorra', sourcetype='government', aggdate='2018-01-25', ismobile=0, value_vec=DenseVector([26.0]), value_scaled=DenseVector([0.0024]), aggdt=datetime.date(2018, 1, 25)), Row(parameter='co', location='AD0942A', value=500.0, city='Escaldes-Engordany', country='AD', sourcename='EEA Andorra', sourcetype='government', aggdate='2018-01-25', ismobile=0, value_vec=DenseVector([500.0]), value_scaled=DenseVector([0.0453]), aggdt=datetime.date(2018, 1, 25)), Row(parameter='so2', location='AD0942A', value=1.0, city='Escaldes-Engordany', country='AD', sourcename='EEA Andorra', sourcetype='government', aggdate='2018-01-25', ismobile=0, value_vec=DenseVector([1.0]), value_scaled=DenseVector([0.0001]), aggdt=datetime.date(2018, 1, 25))]

In [14]:
from pyspark.sql.functions import year, month, quarter, date_format
xform_df = xform_df.withColumn('year',year(xform_df.aggdt)).withColumn('month',month(xform_df.aggdt)).withColumn('quarter',quarter(xform_df.aggdt))
xform_df = xform_df.withColumn("day", date_format(col("aggdt"), "d"))
xform_df.head(3)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(parameter='pm25', location='Smithton', value=1.0, city='Tasmania Region', country='AU', sourcename='Australia - Tasmania', sourcetype='government', aggdate='2018-01-07', ismobile=0, value_vec=DenseVector([1.0]), value_scaled=DenseVector([0.0001]), aggdt=datetime.date(2018, 1, 7), year=2018, month=1, quarter=1, day='7'), Row(parameter='pm10', location='Smithton', value=1.0, city='Tasmania Region', country='AU', sourcename='Australia - Tasmania', sourcetype='government', aggdate='2018-01-07', ismobile=0, value_vec=DenseVector([1.0]), value_scaled=DenseVector([0.0001]), aggdt=datetime.date(2018, 1, 7), year=2018, month=1, quarter=1, day='7'), Row(parameter='pm25', location='Wynyard', value=3.0, city='Tasmania Region', country='AU', sourcename='Australia - Tasmania', sourcetype='government', aggdate='2018-01-07', ismobile=0, value_vec=DenseVector([3.0]), value_scaled=DenseVector([0.0003]), aggdt=datetime.date(2018, 1, 7), year=2018, month=1, quarter=1, day='7')]

In [15]:
# Automatically assign good/bad labels

from pyspark.sql.functions import udf

def isBadAir(v, p):
    if p == 'pm10':
        if v > 50:
            return 1
        else:
            return 0
    elif p == 'pm25':
        if v > 25:
            return 1
        else:
            return 0
    elif p == 'so2':
        if v > 20:
            return 1
        else:
            return 0
    elif p == 'no2':
        if v > 200:
            return 1
        else:
            return 0
    elif p == 'o3':
        if v > 100:
            return 1
        else:
            return 0
    else:
        return 0

isBadAirUdf = udf(isBadAir, IntegerType())

xform_df = xform_df.withColumn('isBadAir', isBadAirUdf('value', 'parameter'))
xform_df.head(3)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(parameter='pm10', location='AD0942A', value=26.0, city='Escaldes-Engordany', country='AD', sourcename='EEA Andorra', sourcetype='government', aggdate='2018-01-25', ismobile=0, value_vec=DenseVector([26.0]), value_scaled=DenseVector([0.0024]), aggdt=datetime.date(2018, 1, 25), year=2018, month=1, quarter=1, day='25', isBadAir=0), Row(parameter='co', location='AD0942A', value=500.0, city='Escaldes-Engordany', country='AD', sourcename='EEA Andorra', sourcetype='government', aggdate='2018-01-25', ismobile=0, value_vec=DenseVector([500.0]), value_scaled=DenseVector([0.0453]), aggdt=datetime.date(2018, 1, 25), year=2018, month=1, quarter=1, day='25', isBadAir=0), Row(parameter='so2', location='AD0942A', value=1.0, city='Escaldes-Engordany', country='AD', sourcename='EEA Andorra', sourcetype='government', aggdate='2018-01-25', ismobile=0, value_vec=DenseVector([1.0]), value_scaled=DenseVector([0.0001]), aggdt=datetime.date(2018, 1, 25), year=2018, month=1, quarter=1, day='25', isBadAir=0

## Categorical encodings

For low-cardinality features, we will use one-hot encoding.  For others we will use index (ordinal) encoding, in the expectation that we will calculate embeddings later on.

In [17]:
from pyspark.ml.feature import OneHotEncoderEstimator

parameter_indexer = StringIndexer(inputCol="parameter", outputCol="indexed_parameter")
location_indexer = StringIndexer(inputCol="location", outputCol="indexed_location")
city_indexer = StringIndexer(inputCol="city", outputCol="indexed_city")
country_indexer = StringIndexer(inputCol="country", outputCol="indexed_country")
sourcename_indexer = StringIndexer(inputCol="sourcename", outputCol="indexed_sourcename")
sourcetype_indexer = StringIndexer(inputCol="sourcetype", outputCol="indexed_sourcetype")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
enc_est = OneHotEncoderEstimator(inputCols=["indexed_parameter"],
                                 outputCols=["vec_parameter"])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
enc_pipeline = Pipeline(stages=[parameter_indexer, location_indexer, city_indexer, country_indexer, sourcename_indexer, sourcetype_indexer, enc_est])
enc_model = enc_pipeline.fit(xform_df)
enc_df = enc_model.transform(xform_df)
enc_df.head(3)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(parameter='pm25', location='Smithton', value=1.0, city='Tasmania Region', country='AU', sourcename='Australia - Tasmania', sourcetype='government', aggdate='2018-01-07', ismobile=0, value_vec=DenseVector([1.0]), value_scaled=DenseVector([0.0001]), aggdt=datetime.date(2018, 1, 7), year=2018, month=1, quarter=1, day='7', isBadAir=0, indexed_parameter=5.0, indexed_location=521.0, indexed_city=12.0, indexed_country=10.0, indexed_sourcename=13.0, indexed_sourcetype=0.0, vec_parameter=SparseVector(6, {5: 1.0})), Row(parameter='pm10', location='Smithton', value=1.0, city='Tasmania Region', country='AU', sourcename='Australia - Tasmania', sourcetype='government', aggdate='2018-01-07', ismobile=0, value_vec=DenseVector([1.0]), value_scaled=DenseVector([0.0001]), aggdt=datetime.date(2018, 1, 7), year=2018, month=1, quarter=1, day='7', isBadAir=0, indexed_parameter=4.0, indexed_location=521.0, indexed_city=12.0, indexed_country=10.0, indexed_sourcename=13.0, indexed_sourcetype=0.0, vec_param

In [42]:
param_cols = enc_df.schema.fields[17].metadata['ml_attr']['vals']
param_cols

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['no2', 'o3', 'so2', 'co', 'pm10', 'pm25', 'bc']

## Split and save

In [31]:
enc_df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- parameter: string (nullable = true)
 |-- location: string (nullable = true)
 |-- value: double (nullable = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)
 |-- sourcename: string (nullable = true)
 |-- sourcetype: string (nullable = true)
 |-- aggdate: string (nullable = true)
 |-- ismobile: integer (nullable = true)
 |-- value_vec: vector (nullable = true)
 |-- value_scaled: vector (nullable = true)
 |-- aggdt: date (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- quarter: integer (nullable = true)
 |-- day: string (nullable = true)
 |-- isBadAir: integer (nullable = true)
 |-- indexed_parameter: double (nullable = false)
 |-- indexed_location: double (nullable = false)
 |-- indexed_city: double (nullable = false)
 |-- indexed_country: double (nullable = false)
 |-- indexed_sourcename: double (nullable = false)
 |-- indexed_sourcetype: double (nullable = false)
 |-- vec_parameter: vector (nulla

In [50]:
final_df = enc_df.drop('parameter').drop('location').drop('city').drop('country').drop('sourcename').drop('sourcetype').drop('aggdate') \
    .drop('value_vec').drop('aggdt').drop('indexed_parameter')
#final_df.head(3)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [51]:
from pyspark.sql.types import DoubleType

firstelement=udf(lambda v:str(v[0]),StringType())
final_df = final_df.withColumn('value_str', firstelement('value_scaled'))
final_df = final_df.withColumn("value",final_df.value_str.cast(DoubleType())).drop('value_str').drop('value_scaled')
#final_df.head(3)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [52]:
from pyspark.ml.linalg import Vectors

def extract(row):
    return (row.value, row.ismobile, row.year, row.month, row.quarter, row.day, row.isBadAir, 
            row.indexed_location, row.indexed_city, row.indexed_sourcename, 
            row.indexed_sourcetype) + tuple(row.vec_parameter.toArray().tolist())

final_df = final_df.rdd.map(extract).toDF(["value", "ismobile", "year", "month", "quarter", "day", "isBadAir",
                               "location", "city", "sourcename", "sourcetype"] + param_cols[:-1])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [53]:
final_df.head(3)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(value=9.061035000521784e-05, ismobile=0, year=2018, month=1, quarter=1, day='13', isBadAir=0, location=521.0, city=12.0, sourcename=13.0, sourcetype=0.0, no2=0.0, o3=0.0, so2=0.0, co=0.0, pm10=0.0, pm25=1.0), Row(value=0.0007248828000417428, ismobile=0, year=2018, month=1, quarter=1, day='13', isBadAir=0, location=521.0, city=12.0, sourcename=13.0, sourcetype=0.0, no2=0.0, o3=0.0, so2=0.0, co=0.0, pm10=1.0, pm25=0.0), Row(value=9.061035000521784e-05, ismobile=0, year=2018, month=1, quarter=1, day='13', isBadAir=0, location=501.0, city=12.0, sourcename=13.0, sourcetype=0.0, no2=0.0, o3=0.0, so2=0.0, co=0.0, pm10=0.0, pm25=1.0)]

In [54]:
(train_df, validation_df, test_df) = final_df.randomSplit([0.7, 0.2, 0.1])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [56]:
train_df.write.option("header",True).csv("s3://rdtest-data/prepared/train/")
test_df.write.option("header",True).csv("s3://rdtest-data/prepared/test/")
validation_df.write.option("header",True).csv("s3://rdtest-data/prepared/validation/")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…